# 🍜 Lesson 1: AI Kenal Gambar!

**Masa:** 60 minit

**Goal:** Buat AI yang boleh kenal perbezaan gambar - collect data sendiri dari internet!

---

## 🤔 Kenapa Belajar Ni?

Korang tau tak:
- **Instagram/TikTok filters** - detect muka korang guna AI
- **Grab Food** - scan gambar makanan untuk categorize
- **CCTV** - detect orang/kereta

Semua ni guna **Image Classification**! Jom buat sendiri! 🔥

### 🎯 Hari Ni Kita Belajar:
1. **Search** gambar dari internet (guna DuckDuckGo)
2. **Download** gambar tu
3. **Clean** data (buang gambar rosak)
4. **Train** AI model!

Sama macam **Jeremy Howard** dalam course.fast.ai! 🚀

> **Note:** DuckDuckGo sometimes rate limits. If search fails, try: Runtime > Restart runtime, then run again.

## Step 1: Setup - Install & Import Libraries

In [ ]:
# Suppress warnings FIRST (before any imports/installs)
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Install required packages
!pip install -Uqq fastai
!pip install -Uqq duckduckgo_search

In [2]:
# Import semua yang kita perlukan
from duckduckgo_search import DDGS
from fastcore.all import *
from fastai.vision.all import *
import time

print("✅ Setup complete! Ready to build AI!")

✅ Setup complete! Ready to build AI!


## Step 2: Buat Function untuk Search Gambar

DuckDuckGo ni search engine macam Google, tapi more privacy-friendly.

Kita buat function untuk search dan download gambar dari internet!

**Note:** Kita add retry logic sebab DuckDuckGo kadang-kadang rate limit kalau terlalu banyak request.

In [3]:
def search_images(term, max_images=30):
    """
    Search images dari DuckDuckGo with retry logic
    
    Args:
        term: Apa nak search (contoh: "nasi lemak")
        max_images: Berapa banyak gambar nak dapat
    
    Returns:
        List of image URLs
    """
    print(f"🔍 Searching for '{term}'...")
    
    # Custom headers to avoid being blocked (from fast.ai forums)
    headers = {
        "Accept-Encoding": "gzip, deflate, br",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
    }
    
    for attempt in range(3):  # Try 3 times
        try:
            # Use headers and longer timeout to avoid blocks
            with DDGS(headers=headers, timeout=30) as ddgs:
                results = list(ddgs.images(term, max_results=max_images))
                urls = L([r['image'] for r in results])
                print(f"   Found {len(urls)} images!")
                return urls
        except Exception as e:
            if 'Ratelimit' in str(e) or '403' in str(e) or 'forbidden' in str(e).lower():
                wait_time = (attempt + 1) * 15  # 15s, 30s, 45s (longer waits)
                print(f"   ⏳ Rate limited, waiting {wait_time}s...")
                time.sleep(wait_time)
            else:
                print(f"   ⚠️ Error: {e}")
                return L([])
    
    print("   ❌ Failed after 3 attempts")
    return L([])

## Step 3: Download Dataset - Malaysian Food! 🍜

Sekarang kita download gambar untuk **3 jenis makanan Malaysia**:
- 🍚 **Nasi Lemak**
- 🥞 **Roti Canai**
- 🍜 **Mee Goreng**

Untuk setiap makanan, kita download ~20 gambar (cukup untuk demo)

In [4]:
# Define makanan yang kita nak classify
searches = ['nasi lemak', 'roti canai', 'mee goreng']

# Create main folder
path = Path('malaysian_food')

# Download gambar untuk setiap jenis makanan
total_downloaded = 0
for search_term in searches:
    # Buat folder untuk setiap makanan
    folder_name = search_term.replace(' ', '_')
    dest = path / folder_name
    dest.mkdir(exist_ok=True, parents=True)
    
    # Search dan download gambar (30 images per category for better training)
    urls = search_images(f"{search_term} food photo", max_images=30)
    
    if len(urls) > 0:
        download_images(dest, urls=urls)
        downloaded = len(list(dest.glob('*')))
        total_downloaded += downloaded
        print(f"✅ {search_term} done! ({downloaded} images)\n")
    else:
        print(f"⚠️ {search_term} - no images found\n")
    
    # PENTING: Wait 10 saat antara setiap search untuk elak rate limit
    print("⏳ Waiting 10s before next search...")
    time.sleep(10)

print("="*50)
if total_downloaded > 0:
    print(f"🎉 DOWNLOAD COMPLETE! Total: {total_downloaded} images")
else:
    print("⚠️ DuckDuckGo blocked - this sometimes happens in Colab")
    print("💡 Try: Runtime > Restart runtime, then run again")
print("="*50)

🔍 Searching for 'nasi lemak food photo'...


C:\Users\tengk\AppData\Local\Temp\ipykernel_36112\236100525.py:24: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS(headers=headers, timeout=30) as ddgs:


   Found 30 images!
✅ nasi lemak done! (30 images)

⏳ Waiting 10s before next search...
🔍 Searching for 'roti canai food photo'...


C:\Users\tengk\AppData\Local\Temp\ipykernel_36112\236100525.py:24: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS(headers=headers, timeout=30) as ddgs:


   ⏳ Rate limited, waiting 15s...


...

   ⏳ Rate limited, waiting 15s...


C:\Users\tengk\AppData\Local\Temp\ipykernel_36112\236100525.py:24: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS(headers=headers, timeout=30) as ddgs:


   ⏳ Rate limited, waiting 30s...


C:\Users\tengk\AppData\Local\Temp\ipykernel_36112\236100525.py:24: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS(headers=headers, timeout=30) as ddgs:


   ⏳ Rate limited, waiting 45s...
   ❌ Failed after 3 attempts
⚠️ mee goreng - no images found

⏳ Waiting 10s before next search...
🎉 DOWNLOAD COMPLETE! Total: 30 images


## Step 4: Clean Data - Buang Gambar Rosak

Kadang-kadang gambar yang download tu rosak atau tak boleh dibuka.

Fastai ada function `verify_images()` untuk check dan buang gambar yang tak valid.

In [ ]:
# Check and clean images
image_files = get_image_files(path)

# Clean corrupted images
if len(image_files) > 0:
    failed = verify_images(image_files)
    failed.map(Path.unlink)
    print(f"🗑️ Removed {len(failed)} corrupted/invalid images")
    image_files = get_image_files(path)  # Refresh after cleaning

print(f"📁 Total images: {len(image_files)}")

# Check if we have enough images for training
if len(image_files) < 10:
    print("\n⚠️ Not enough images for training!")
    print("💡 Solutions:")
    print("   1. Runtime > Restart runtime, then run cells again")
    print("   2. Try again later (DuckDuckGo rate limits reset)")
    print("   3. Manually download images to the folders")
else:
    print("✅ Ready for training!")

## Step 5: Tengok Sample Gambar

Jom tengok gambar yang kita download!

In [ ]:
# Tengok beberapa gambar dari dataset
from PIL import Image as PILImg
import matplotlib.pyplot as plt

image_files = get_image_files(path)

if len(image_files) > 0:
    # Show up to 6 sample images
    n_show = min(6, len(image_files))
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    axes = axes.flatten()
    
    for idx in range(n_show):
        img = PILImg.open(image_files[idx])
        axes[idx].imshow(img)
        axes[idx].set_title(image_files[idx].parent.name if image_files[idx].parent.name != 'images' else image_files[idx].name[:20])
        axes[idx].axis('off')
    
    # Hide unused subplots
    for idx in range(n_show, 6):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No images to display yet")

## Step 6: Sediakan Data untuk Training (DataLoaders)

### 🧠 Concept: DataBlock

DataBlock ni macam **recipe** untuk prepare data:
- `blocks=(ImageBlock, CategoryBlock)` - Input gambar, output category
- `get_items=get_image_files` - Ambil semua gambar dari folder
- `splitter=RandomSplitter` - Split data untuk train/validation
- `get_y=parent_label` - Label = nama folder (nasi_lemak, roti_canai, etc.)
- `item_tfms=Resize(192)` - Resize gambar jadi 192x192

In [ ]:
# Buat DataLoaders untuk Malaysian Food
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(192)
).dataloaders(path, bs=8, num_workers=0)  # Small batch size, num_workers=0 for Colab

print("✅ DataLoaders created!")
print(f"   Training batches: {len(dls.train)}")
print(f"   Validation batches: {len(dls.valid)}")
print(f"   Categories: {dls.vocab}")

In [ ]:
# Jom tengok data kita!
dls.show_batch(max_n=9)

## Step 7: Buat Model (AI Brain! 🧠)

### 🧠 Concept: Transfer Learning

Bayangkan macam ni:
- **ResNet18** ni dah "belajar" dari **jutaan** gambar
- Dah tau macam mana nak kenal edges, shapes, textures
- Kita just "fine-tune" untuk task kita je

Macam **copy homework kawan yang dah pandai**, lepas tu edit sikit! 😂

In [ ]:
# Buat learner (model + training settings)
# Guna resnet18 - smaller & faster untuk demo
learn = vision_learner(dls, resnet18, metrics=error_rate)

print("🧠 Model created! Ready to train!")

## Step 8: TRAIN! 🏋️

Ni part paling best - tengok AI belajar secara real-time!

Kita train untuk **3 epochs** (3 rounds through all images)

In [ ]:
# Train untuk 3 epochs (no_bar() untuk Colab compatibility)
with learn.no_bar():
    learn.fine_tune(3)

### 🎉 Check Results!

Tengok **error_rate** - kalau rendah, meaning AI kita accurate!

Contoh: error_rate = 0.15 means **85% accurate**!

## Step 9: Tengok Predictions

Jom tengok macam mana AI kita perform!

In [ ]:
# Show predictions vs actual
learn.show_results(max_n=6)

In [ ]:
# Confusion Matrix - nak tengok mana yang AI paling confuse
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

## Step 10: Test dengan Gambar dari Dataset

Test AI dengan gambar dari dataset kita!

In [ ]:
# Test dengan gambar dari dataset
test_files = get_image_files(path)
if len(test_files) > 0:
    # Pick random image
    import random
    test_img = PILImage.create(random.choice(test_files))
    
    # Predict!
    pred, pred_idx, probs = learn.predict(test_img)
    
    print(f"🔮 AI kata: {str(pred).upper()}")
    print(f"📊 Confidence: {probs[pred_idx]:.1%}")
    test_img.to_thumb(256, 256)
else:
    print("⚠️ No test images available")

---

# 🏆 CHALLENGE TIME!

## Buat Classifier Korang Sendiri!

Guna code template di bawah untuk buat classifier untuk topic yang korang suka:

### Ideas:
1. **Kereta Malaysia** - Proton vs Perodua vs Honda
2. **K-pop Groups** - BLACKPINK vs BTS vs TWICE  
3. **Football Teams** - JDT vs Selangor vs Perak
4. **Animals** - Bird vs Cat vs Dog
5. **Your own idea!** - Be creative! 🎨

In [ ]:
# === TEMPLATE: Buat Classifier Sendiri! ===

# 1. Define categories korang
my_searches = ['bird photo', 'cat photo', 'dog photo']  # <-- TUKAR NI!

# 2. Create folder
my_path = Path('my_classifier')

# 3. Download images (dengan delay untuk elak rate limit)
for search_term in my_searches:
    folder_name = search_term.split()[0]  # First word as folder name
    dest = my_path / folder_name
    dest.mkdir(exist_ok=True, parents=True)
    
    urls = search_images(search_term, max_images=20)
    if len(urls) > 0:
        download_images(dest, urls=urls)
    
    time.sleep(5)  # Wait 5s between searches!
    print(f"✅ {folder_name} done!")

# 4. Clean data
failed = verify_images(get_image_files(my_path))
failed.map(Path.unlink)
print(f"\n📁 Total images: {len(get_image_files(my_path))}")

In [ ]:
# 5. Create DataLoaders
my_dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(192)
).dataloaders(my_path, bs=32, num_workers=0)  # num_workers=0 for Colab

# 6. Show sample
my_dls.show_batch(max_n=9)

In [ ]:
# 7. Train model! (no_bar() for Colab compatibility)
my_learn = vision_learner(my_dls, resnet18, metrics=error_rate)
with my_learn.no_bar():
    my_learn.fine_tune(3)

In [ ]:
# 8. Check results
my_learn.show_results(max_n=6)

---

## 💡 Discussion Questions

1. **Kenapa AI kadang-kadang confuse?**
   - Gambar dari internet kadang-kadang tak berkualiti
   - Background/lighting yang sama
   - Need more training data

2. **Macam mana nak improve accuracy?**
   - Download lebih banyak gambar (100+ setiap category)
   - Manually check dan buang gambar yang tak betul
   - Train lebih lama (5+ epochs)
   - Guna model lebih besar (resnet34, resnet50)

3. **Workflow untuk collect custom data:**
   ```
   1. search_images() → Dapat URLs
   2. download_images() → Download ke folder
   3. verify_images() → Buang yang rosak
   4. DataBlock → Sediakan untuk training
   5. vision_learner() → Train!
   ```

---

## 📚 Resources

- **course.fast.ai** - Full free deep learning course by Jeremy Howard
- **Kaggle.com** - Download datasets dan compete

---

*"Sekarang korang dah boleh train AI dengan data sendiri dari internet! That's powerful!"* 🚀🇲🇾